### Temp - Will be convereted into main.py once complete
Currently being used for testing

In [40]:
import torch
import pickle
import os
import numpy as np

from data.process_data import load_data, augment_data, preprocess_data, to_tensors, split_batch
from models.quickdraw_cnn import QuickDrawCNN_V1, QuickDrawCNN_V2
from training.model_train import train

In [41]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")
image_dir = "data/numpy_bitmap/"

Device: cuda


In [42]:
categories = os.listdir(image_dir)
p1, p2 = categories[:len(categories)//2], categories[len(categories)//2:]

In [43]:
features_p1, labels_p1 = load_data(p1, image_dir, file_standardize=False, sample=2)
features_p2, labels_p2 = load_data(p2, image_dir, file_standardize=False, sample=2)

Loading 172 categories...


Loading data: 100%|██████████| 2/2 [00:00<00:00, 46.14it/s]


Data loaded, size: (249646, 784)
Loading 173 categories...


Loading data: 100%|██████████| 2/2 [00:00<00:00, 48.93it/s]

Data loaded, size: (281931, 784)


In [45]:
# Preprocessing data split array -> 28x28 numpy
features_p1, labels_p1 = preprocess_data(features_p1, labels_p1)
features_p2, labels_p2 = preprocess_data(features_p2, labels_p2)

Preprocessing 2 images...


Processing labels: 100%|██████████| 2/2 [00:00<?, ?it/s]


Preprocessing 2 images...


Processing labels: 100%|██████████| 2/2 [00:00<?, ?it/s]


In [46]:
##################################
labels_p1, labels_p2

(['arm', 'bench'], ['peanut', 'grapes'])

In [47]:
#### ERROR –> Since they are run seperately, the labels_map is the same even though its different labels

features_p1, labels_p1,  labels_map_p1 = to_tensors(features_p1, labels_p1)
features_p2, labels_p2,  labels_map_p2 = to_tensors(features_p2, labels_p2)

Converting 2 images to tensors...
Features shape: torch.Size([249646, 1, 28, 28]), Labels shape: torch.Size([2])
Converting 2 images to tensors...
Features shape: torch.Size([281931, 1, 28, 28]), Labels shape: torch.Size([2])


In [48]:
##################################
labels_p1, labels_p2

(tensor([0, 1]), tensor([1, 0]))

In [10]:
# Split further
features_p1_1, features_p1_2 = features_p1[:len(features_p1) // 2], features_p1[len(features_p1) // 2:]
labels_p1_1, labels_p1_2 = labels_p1[:len(labels_p1) // 2], labels_p1[len(labels_p1) // 2:]
features_p2_1, features_p2_2 = features_p2[:len(features_p2) // 2], features_p2[len(features_p2) // 2:]
labels_p2_1, labels_p2_2 = labels_p2[:len(labels_p2) // 2], labels_p2[len(labels_p2) // 2:]

# Augment data
features_p1_1, labels_p1_1 = augment_data(features_p1_1, labels_p1_1, rot=True, angle=15, h_flip=False)
features_p1_2, labels_p1_2 = augment_data(features_p1_2, labels_p1_2, rot=True, angle=15, h_flip=False)
features_p2_1, labels_p2_1 = augment_data(features_p2_1, labels_p2_1, rot=True, angle=15, h_flip=False)
features_p2_2, labels_p2_2 = augment_data(features_p2_2, labels_p2_2, rot=True, angle=15, h_flip=False)

Starting size: (181426, 1, 28, 28)
Rotating images by 15 degrees...


Rotating images...: 100%|██████████| 181426/181426 [00:41<00:00, 4420.49it/s]


Augmented size: 
Starting size: (181427, 1, 28, 28)
Rotating images by 15 degrees...


Rotating images...: 100%|██████████| 181427/181427 [00:41<00:00, 4392.35it/s]


Augmented size: 
Starting size: torch.Size([60475, 1, 28, 28])
Rotating images by 15 degrees...


Rotating images...: 100%|██████████| 60475/60475 [00:14<00:00, 4260.03it/s]


Augmented size: 
Starting size: torch.Size([60476, 1, 28, 28])
Rotating images by 15 degrees...


Rotating images...: 100%|██████████| 60476/60476 [00:14<00:00, 4270.55it/s]


Augmented size: 


In [15]:
# Combine features and labels
features = np.concatenate((features_p1_1, features_p1_2, features_p2_1, features_p2_2), axis=0)
labels = np.concatenate((labels_p1_1, labels_p1_2, labels_p2_1, labels_p2_2), axis=0)

In [11]:


# Split into train, test, and validation sets
train_loader, test_loader, val_loader = split_batch(features, labels, batch_size=64)

TypeError: split_batch() got multiple values for argument 'batch_size'

In [ ]:
model = QuickDrawCNN_V2().to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
train_loss, val_loss, train_acc, val_acc = train(model = model,
                                                  train_loader = train_loader,
                                                  val_loader = val_loader,
                                                  epochs = 10,
                                                  criterion = criterion,
                                                  optimizer = optimizer,
                                                  device = device)   

In [ ]:
save_path = "models/quickdraw_cnn_v2.pth"
torch.save(model.state_dict(), save_path)
print(f"Model saved to {save_path}.")

In [ ]:
with open("models/labels_map.pkl", "wb") as f:
    pickle.dump(labels_map, f)
    pickle.dump(train_loss, f)
    pickle.dump(val_loss, f)
    pickle.dump(train_acc, f)
    pickle.dump(val_acc, f)